In [2]:
%showtypes on

Types will be printed.


In [3]:
case class Flight(DEST_COUNTRY_NAME: String,
                  ORIGIN_COUNTRY_NAME: String, 
                  count: BigInt)

defined class Flight


In [4]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_9054: Some(http://gw02.itversity.com:4048)

In [5]:
val flightsDF = spark.
                read.
                parquet("/user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/")

Waiting for a Spark session to start...

flightsDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [22]:
val flights = flightsDF.as[Flight].cache()

flights = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [23]:
println(flightsDF.getClass().getName)
println(flights.getClass().getName)

org.apache.spark.sql.Dataset
org.apache.spark.sql.Dataset


In [24]:
flights.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [25]:
flights.first.DEST_COUNTRY_NAME

United States

In [26]:
def originIsDestination(flight_row: Flight): Boolean = {
  return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}

originIsDestination: (flight_row: Flight)Boolean


In [27]:
flights.filter(x => originIsDestination(x)).collect()

[Flight(United States,United States,348113)]

In [28]:
flights.collect().filter(x => originIsDestination(x))

[Flight(United States,United States,348113)]

In [29]:
flights.filter(x => originIsDestination(x)).show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|348113|
+-----------------+-------------------+------+



In [30]:
val destinations = flights.map(f => f.DEST_COUNTRY_NAME)

destinations = [value: string]


[value: string]

In [31]:
val localDestinations = destinations.take(5)

localDestinations = Array(United States, United States, United States, Egypt, Equatorial Guinea)


[United States, United States, United States, Egypt, Equatorial Guinea]

In [32]:
case class FlightMetadata(count: BigInt, randomData: BigInt)

defined class FlightMetadata


In [36]:
spark.range(500).map(x => (x, scala.util.Random.nextLong)).show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0| 8797984346824714710|
|  1| 9146943581127446671|
|  2|-3816535872406726930|
|  3|-8655133167090689937|
|  4|-1125574671320092950|
|  5| -731329737330901770|
|  6| 6493083979319074284|
|  7|-6995937290632452009|
|  8|-7274215871035376116|
|  9|  556681772115901767|
| 10| 3157650557017053308|
| 11| 2202492472679610047|
| 12| 3910438635826861293|
| 13|-1226799466950852608|
| 14|-1334646806594159823|
| 15| 2077318981243930839|
| 16|-3524326559222748382|
| 17|-5464992708199015902|
| 18|-5952215271453299007|
| 19| 8137380566034050706|
+---+--------------------+
only showing top 20 rows



In [37]:
val flightsMeta1 = spark.range(500).map(x => (x, scala.util.Random.nextLong))
  .withColumnRenamed("_1", "count").withColumnRenamed("_2", "randomData")

flightsMeta1 = [count: bigint, randomData: bigint]


[count: bigint, randomData: bigint]

In [38]:
flightsMeta1.show()

+-----+--------------------+
|count|          randomData|
+-----+--------------------+
|    0|-5222736348851663315|
|    1| 5280068547074691804|
|    2|-5006658480135877938|
|    3|  246838347996943929|
|    4| 1522855005557778159|
|    5| 8177965701404726124|
|    6| 7947173060641809733|
|    7| 3885003387053804639|
|    8| 5440324391055950483|
|    9| 6812969115974481843|
|   10| 7487278294913786887|
|   11|-8316462468292808770|
|   12|-7178179202350921756|
|   13|-8856298219954500137|
|   14|-4295637448613503904|
|   15|-6164241860741630512|
|   16|-6720407068351684401|
|   17|-3384102449598802493|
|   18|-7681307719970982656|
|   19|-2371163396209271279|
+-----+--------------------+
only showing top 20 rows



In [46]:
val flightsMeta = flightsMeta1.as[FlightMetadata]

flightsMeta = [count: bigint, randomData: bigint]


[count: bigint, randomData: bigint]

In [47]:
val flights2 = flights.joinWith(flightsMeta, flights.col("count") === flightsMeta.col("count"))

flights2 = [_1: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field>, _2: struct<count: bigint, randomData: bigint>]


[_1: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field>, _2: struct<count: bigint, randomData: bigint>]

In [56]:
flights2.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[United States, R...|[1, 7486920142528...|
|[United States, I...|[264, 14887601199...|
|[United States, I...|[69, 512362281235...|
|[Egypt, United St...|[24, -85075162359...|
|[Equatorial Guine...|[1, 7486920142528...|
|[United States, S...|[25, 399857725670...|
|[United States, G...|[54, -14531997253...|
|[Costa Rica, Unit...|[477, 48287764903...|
|[Senegal, United ...|[29, -20906581262...|
|[United States, M...|[44, 472327486055...|
|[Guyana, United S...|[17, -71481597569...|
|[United States, S...|[53, 532571079948...|
|[Malta, United St...|[1, 7486920142528...|
|[Bolivia, United ...|[46, 115383893391...|
|[Anguilla, United...|[21, -83428689467...|
|[Turks and Caicos...|[136, -1717177994...|
|[United States, A...|[2, -652902625537...|
|[Saint Vincent an...|[1, 7486920142528...|
|[Italy, United St...|[390, 72596013790...|
|[United States, R...|[156, -216

In [53]:
flights2.take(10).foreach(println)

(Flight(United States,Romania,1),FlightMetadata(1,-2436322164652305444))
(Flight(United States,Ireland,264),FlightMetadata(264,-1517365464521077716))
(Flight(United States,India,69),FlightMetadata(69,-8615909625094518161))
(Flight(Egypt,United States,24),FlightMetadata(24,5250523965836945935))
(Flight(Equatorial Guinea,United States,1),FlightMetadata(1,-2436322164652305444))
(Flight(United States,Singapore,25),FlightMetadata(25,-3100869301626318215))
(Flight(United States,Grenada,54),FlightMetadata(54,1822674138899312223))
(Flight(Costa Rica,United States,477),FlightMetadata(477,7053284717368381739))
(Flight(Senegal,United States,29),FlightMetadata(29,-903613913428713617))
(Flight(United States,Marshall Islands,44),FlightMetadata(44,-4981955470875250154))


In [55]:
flights2.selectExpr("_1.DEST_COUNTRY_NAME").show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|       United States|
|       United States|
|               Egypt|
|   Equatorial Guinea|
|       United States|
|       United States|
|          Costa Rica|
|             Senegal|
|       United States|
|              Guyana|
|       United States|
|               Malta|
|             Bolivia|
|            Anguilla|
|Turks and Caicos ...|
|       United States|
|Saint Vincent and...|
|               Italy|
|       United States|
+--------------------+
only showing top 20 rows



In [60]:
flights2.selectExpr("_1.DEST_COUNTRY_NAME").take(4)

United States
United States
United States
Egypt


In [57]:
val flights3 = flights.join(flightsMeta, Seq("count"))

flights3 = [count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]


[count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]

In [61]:
flights3.take(4)

1,United States,Romania,-7596727126198596367
264,United States,Ireland,-613671581099459947
69,United States,India,-880858559134314524
24,Egypt,United States,5919292528565860569


In [ ]:
val flights4 = flights.join(flightsMeta.toDF(), Seq("count"))

In [62]:
flights4.take(4)

1,United States,Romania,-8729287246109974057
264,United States,Ireland,4726576919667245594
69,United States,India,4524141979811486012
24,Egypt,United States,7088040882726746808


In [65]:
flights4.show()

+-----+--------------------+-------------------+--------------------+
|count|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|          randomData|
+-----+--------------------+-------------------+--------------------+
|    1|       United States|            Romania| 4214651092576403978|
|  264|       United States|            Ireland|-8818261377422385763|
|   69|       United States|              India|-4720274176553143199|
|   24|               Egypt|      United States|-3234539002274696978|
|    1|   Equatorial Guinea|      United States| 4214651092576403978|
|   25|       United States|          Singapore| 6751583061209774868|
|   54|       United States|            Grenada| 3432990127083356520|
|  477|          Costa Rica|      United States|  413833460887673837|
|   29|             Senegal|      United States| 8331522646064446304|
|   44|       United States|   Marshall Islands|-4103919259039256814|
|   17|              Guyana|      United States|-7198676132565158101|
|   53|       United

In [64]:
flights.join(flightsMeta.toDF(), "count").show()

+-----+--------------------+-------------------+--------------------+
|count|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|          randomData|
+-----+--------------------+-------------------+--------------------+
|    1|       United States|            Romania|  494087802871862969|
|  264|       United States|            Ireland|-1843332084659968982|
|   69|       United States|              India| 6767226877438942883|
|   24|               Egypt|      United States| 8902839180265810567|
|    1|   Equatorial Guinea|      United States|  494087802871862969|
|   25|       United States|          Singapore|-8947056371109695183|
|   54|       United States|            Grenada| 3866869310290976879|
|  477|          Costa Rica|      United States|-8712358538864828642|
|   29|             Senegal|      United States|-6057149665589125323|
|   44|       United States|   Marshall Islands|-4184943351838451479|
|   17|              Guyana|      United States|-6702928344961400592|
|   53|       United

In [71]:
flights.groupBy("DEST_COUNTRY_NAME").count().take(10)

Anguilla,1
Russia,1
Paraguay,1
Senegal,1
Sweden,1
Kiribati,1
Guyana,1
Philippines,1
Malaysia,1
Singapore,1


In [69]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [72]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().explain

== Physical Plan ==
*(2) HashAggregate(keys=[value#687], functions=[count(1)])
+- Exchange hashpartitioning(value#687, 200)
   +- *(1) HashAggregate(keys=[value#687], functions=[partial_count(1)])
      +- *(1) Project [value#687]
         +- AppendColumns <function1>, newInstance(class $line21.$read$$iw$$iw$Flight), [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#687]
            +- InMemoryTableScan [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L]
                  +- InMemoryRelation [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- *(1) FileScan parquet [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [],

In [73]:
def grpSum(countryName:String, values: Iterator[Flight]) = {
  values.dropWhile(_.count < 5).map(x => (countryName, x))
}

grpSum: (countryName: String, values: Iterator[Flight])Iterator[(String, Flight)]


In [75]:
flights.take(10)

[Flight(United States,Romania,1), Flight(United States,Ireland,264), Flight(United States,India,69), Flight(Egypt,United States,24), Flight(Equatorial Guinea,United States,1), Flight(United States,Singapore,25), Flight(United States,Grenada,54), Flight(Costa Rica,United States,477), Flight(Senegal,United States,29), Flight(United States,Marshall Islands,44)]

In [82]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).flatMapGroups(grpSum).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Paraguay,Flight(Paraguay,United States,90)), (Russia,Flight(Russia,United States,152)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Singapore,Flight(Singapore,United States,25)), (Fiji,Flight(Fiji,United States,53))]

In [83]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [92]:
flights.sort("DEST_COUNTRY_NAME").take(10)

[Flight(Afghanistan,United States,11), Flight(Angola,United States,14), Flight(Anguilla,United States,21), Flight(Antigua and Barbuda,United States,123), Flight(Argentina,United States,184), Flight(Aruba,United States,359), Flight(Australia,United States,290), Flight(Austria,United States,36), Flight(Azerbaijan,United States,1), Flight(Bahrain,United States,30)]

In [93]:
def grpSum2(f:Flight):Integer = {
  1
}

grpSum2: (f: Flight)Integer


In [94]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).mapValues(grpSum2).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [104]:
def sum2(left:Flight, right:Flight) = {
 Flight(null, null, left.count + right.count)
}
def sum3(left:Flight, right:Flight) = {
 Flight(null, null, left.count - right.count)
}
def sum4(left:Flight, right:Flight) = {
 Flight(null, null, left.count * right.count)
}

sum2: (left: Flight, right: Flight)Flight
sum3: (left: Flight, right: Flight)Flight
sum4: (left: Flight, right: Flight)Flight


In [105]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [106]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [107]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum3(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [101]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).count()

125

In [102]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().count()

125

In [103]:
flights.groupBy("DEST_COUNTRY_NAME").count().explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#0, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[partial_count(1)])
      +- InMemoryTableScan [DEST_COUNTRY_NAME#0]
            +- InMemoryRelation [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
                  +- *(1) FileScan parquet [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>
